Map both paired and unpaired reads.

In [ ]:
for dir in $(ls); do cd $dir; /local/home/sarahgw/anaconda2/bin/bowtie2 --very-sensitive-local -p 12 -x /local/home/sarahgw/references/Myotis_Lucifugus2.0/MyoLuc_toplevel_index -1 $(pwd)/"$dir"_FU1_trim_p.tagged_pfilter.fastq -2 $(pwd)/"$dir"_FU2_trim_p.tagged_pfilter.fastq -U $(pwd)/"$dir"_FU1_trim_up.tagged_filter.fastq,$(pwd)/"$dir"_FU2_trim_up.tagged_filter.fastq -S $(pwd)/"$dir"_pairedplus_eg2.sam; cd ..; done

      1094025 (56.42%) aligned >1 times
  533990 (6.86%) were unpaired; of these:
    22751 (4.26%) aligned 0 times
    279666 (52.37%) aligned exactly 1 time
    231573 (43.37%) aligned >1 times
97.45% overall alignment rate
11459489 reads; of these:
  10184983 (88.88%) were paired; of these:
    2499945 (24.55%) aligned concordantly 0 times
    5355705 (52.58%) aligned concordantly exactly 1 time
    2329333 (22.87%) aligned concordantly >1 times
    ----
    2499945 pairs aligned concordantly 0 times; of these:
      1067616 (42.71%) aligned discordantly 1 time
    ----
    1432329 pairs aligned 0 times concordantly or discordantly; of these:
      2864658 mates make up the pairs; of these:
        581756 (20.31%) aligned 0 times
        702292 (24.52%) aligned exactly 1 time
        1580610 (55.18%) aligned >1 times
  1274506 (11.12%) were unpaired; of these:
    51583 (4.05%) aligned 0 times
    665468 (52.21%) aligned exactly 1 time
    557455 (43.74%) aligned >1 times
97.07% overall alignment rate
16543246 reads; of these:
  11787792 (71.25%) were paired; of these:
    1926558 (16.34%) aligned concordantly 0 times
    6571409 (55.75%) aligned concordantly exactly 1 time
    3289825 (27.91%) aligned concordantly >1 times
    ----
    1926558 pairs aligned concordantly 0 times; of these:
      788708 (40.94%) aligned discordantly 1 time
    ----
    1137850 pairs aligned 0 times concordantly or discordantly; of these:
      2275700 mates make up the pairs; of these:
        585052 (25.71%) aligned 0 times
        532601 (23.40%) aligned exactly 1 time
        1158047 (50.89%) aligned >1 times
  4755454 (28.75%) were unpaired; of these:
    167802 (3.53%) aligned 0 times
    2562632 (53.89%) aligned exactly 1 time
    2025020 (42.58%) aligned >1 times
97.34% overall alignment rate

Sort sam, convert to bam

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar SortSam \
      I=$(pwd)/"$dir"_pairedplus_eg2.sam \
      O=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      SORT_ORDER=coordinate; cd ..; done

Remove duplicates

In [ ]:
for dir in $(ls); do cd $dir; java -jar /local/home/sarahgw/picard.jar MarkDuplicates \
      I=$(pwd)/"$dir"_pairedplus_eg2_sorted.bam \
      O=$(pwd)/"$dir"_pairedplus_marked_dupes.bam \
      M=$(pwd)/"$dir"_pairedplus_marked_dup_metrics.txt \
      REMOVE_DUPLICATES=true \
      ASSUME_SORT_ORDER=coordinate; cd ..; done 

In [4]:
!head /local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_04/NJ_E_04_pairedplus_marked_dup_metrics.txt

## htsjdk.samtools.metrics.StringHeader
# picard.sam.markduplicates.MarkDuplicates INPUT=[/local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_04/NJ_E_04_pairedplus_eg2_sorted.bam] OUTPUT=/local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_04/NJ_E_04_pairedplus_marked_dupes.bam METRICS_FILE=/local/home/sarahgw/WGS/Sequences/Pool3/NJ_E_04/NJ_E_04_pairedplus_marked_dup_metrics.txt REMOVE_DUPLICATES=true ASSUME_SORT_ORDER=coordinate    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 REMOVE_SEQUENCING_DUPLICATES=false TAGGING_POLICY=DontTag ASSUME_SORTED=false DUPLICATE_SCORING_STRATEGY=SUM_OF_BASE_QUALITIES PROGRAM_RECORD_ID=MarkDuplicates PROGRAM_GROUP_NAME=MarkDuplicates READ_NAME_REGEX=<optimized capture of last three ':' separated fields as numeric values> OPTICAL_DUPLICATE_PIXEL_DISTANCE=100 VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=fal

Calculate coverage stats using bedtools

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_Coverage.txt; cd ..; done

In [ ]:
for dir in $(ls); do cd $dir; samtools view -b $(pwd)/"$dir"_pairedplus_marked_dupes.bam | genomeCoverageBed -d -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>$(pwd)/"$dir"_BaseCoverage.txt; cd ..; done 

In [ ]:
samtools view -b NJ_E_25_pairedplus_marked_dupes.bam | genomeCoverageBed -d -ibam stdin -g /local/home/sarahgw/references/Myotis_Lucifugus2.0/Myoluc_genomeFile.txt>NJ_E_25_BaseCoverage.txt

In [ ]:
for dir in $(ls); do cd $dir;  sed "s/$/\t"$dir"/" "$dir"_Coverage.txt > "$dir"_Coverage_name.txt ; cd ..; done

cat coverage together

In [ ]:
cat ./*/*_Coverage.txt >> ./Coverage_all.txt